## 카운트벡터, TFIDF, SVD이용한 유사도 비교 

In [24]:
import pandas as pd
from pandas import Series, DataFrame

import konlpy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import AgglomerativeClustering

In [25]:
data=pd.read_csv("./Data/article_df_final.csv",encoding='cp949')
df=data.copy()
df=df.loc[:,['title','date','main','section']]
df.head(2)
df=DataFrame(df)
print(df.shape)
df.head(2)


(1500, 4)


,title,date,main,section
0,오후 국회 본회의…'전세사기 특별법'·'김남국 방지법' 처리?,2023.05.25. 오전 9:24,전세사기 특별법과 이른바 '김남국 방지법'으로 불리는 가상자산(코인) 관련 개정안이...,100
1,"이준석, 학력 위조 의혹에 ""하버드 졸업 10억 내기하자…이번엔 박멸""",2023.05.25. 오후 3:53,"""해당 의혹 제기한 유튜버들 싸그리 고소""\n\n\n\n이준석 전 국민의힘 대표가 ...",100


In [26]:
print('열의 결측값의 수:',df.isnull().sum())

열의 결측값의 수: title      0
date       0
main       0
section    0
dtype: int64


In [27]:
df.drop_duplicates(subset=['main'], inplace=True)#중복제거

print('뉴스 기사의 개수: ',len(df))

뉴스 기사의 개수:  1500


In [28]:
#df = df[df.main.str.len() > 100]
df.shape
print(df.groupby('section').size().reset_index(name = 'count'))

   section  count
0      100    250
1      101    250
2      102    250
3      103    250
4      104    250
5      105    250


In [29]:
from konlpy.tag import Okt #konlpy에서 Twitter 형태소 분석기를 import
twitter_tag = Okt()

def my_tokenizer(doc):
    words= [token for token, pos in twitter_tag.pos(doc) if pos in ['Noun'] if len(token) > 1]            
    stopwords = ['올해','오늘','시즌','역시','라며','지난해','이전','기준','최근','현재','회장','서비스','이후','오전','오후','대해','위해','내용','연합뉴스','때문','무단','따른','금지','전재','경향신문','기자','는데','파이낸셜','저작','뉴스','이번','대한','대비']

    words = [word for word in words if not word in stopwords]

    return  words

print("나만의 토크나이저 결과:",my_tokenizer(df.main[1])) 

나만의 토크나이저 결과: ['해당', '의혹', '제기', '유튜', '버들', '고소', '이준석', '국민', '대표', '제주시', '애월읍', '브루클린', '제주', '카페', '독자', '만남', '대표', '이준석', '거부', '미래', '발간', '이준석', '국민', '대표', '갑자기', '자신', '학력', '논란', '내기', '대응', '예고', '대표', '자신', '페이스북', '통해', '자신', '학력', '논란', '기사', '공유', '반복', '타진요', '놀이', '비판', '타진', '요란', '타블로', '진실', '요구', '단체', '약칭', '단체', '회원', '가수', '타블로', '학력', '위조', '의혹', '제기', '허위', '사실', '유포', '실형', '선고', '대표', '타진요', '언급', '이유', '자신', '의혹', '허위', '의혹', '강조', '여러분', '성적표', '졸업', '증명서', '졸업생', '사이트', '접속', '인증', '기관', '결론', '사안', '논문', '표절', '독창', '연구', '강조', '보수', '유튜버들', '하버드', '복수', '전공', '학력', '위조', '의혹', '제기', '대표', '하버드', '졸업', '거짓', '거나', '컴퓨터공학', '경제학', '복수', '전공', '허위', '여부', '내기', '허위', '이면', '여러분', '제안', '해당', '의혹', '제기', '유튜버들', '고소장', '도달', '정신', '박멸', '허위', '사실', '유포자', '제보', '메일', '주소', '공유', '다른', '게시', '패턴', '바보', '어차피', '패턴', '미리', '공유', '과거', '해명', '공유', '게시', '추가', '자꾸', '발급', '성적표', '성적표', '공유', '다른', '게시', '통해', '패턴', '바보', '어차피', '패턴', '미리', '공유', '과거', '학력', '

In [30]:
#BOW
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

#train
cv = CountVectorizer( tokenizer=my_tokenizer)
df_DTM = cv.fit_transform(df.main) #review를 이용하여 count vector를 학습하고, 변환

print('#첫 문서의 내용:', df.main[260])
print('#첫 문서의 카테고리:', df.section[260])

#test
source= df.main[260]
source_cv= cv.transform([source])
#유사도 비교
sim_result_cv= cosine_similarity(source_cv, df_DTM) 

C:\ProgramData\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


#첫 문서의 내용: ▲ 이창용 총재, 금융통화위원회 주재 사진 : 연합뉴스?한국은행 금융통화위원회(금통위)가 오늘(25일) 오전 9시 통화정책방향 회의를 열고 현재 3.50%인 기준금리의 조정 여부를 결정할 예정입니다. 경제·금융 전문가들 사이에서는 금통위가 기준금리를 2월과 4월에 이어 다시 동결할 것이라는 전망이 압도적으로 우세합니다. 지난달 소비자물가 상승률(3.7%)이 14개월 만에 3%대로 떨어져 인플레이션 압력이 다소 줄어든 상태에서, 굳이 한은이 무리한 금리 인상으로 가뜩이나 위축된 경기에 찬물을 끼얹을 이유가 없다는 분석입니다.수출 부진이 이어지는 가운데 지난 1분기 우리나라 실질 국내총생산(GDP) 성장률은 민간소비 덕에 겨우 두 분기 연속 역성장을 피했고, 3월 경상수지도 국내기업 해외 현지법인의 배당에 기대 겨우 석 달 연속 적자를 모면했습니다.하지만 통관기준 무역수지는 4월(-26억 2천만 달러)까지 14개월째 적자에서 벗어나지 못하고 있습니다.이런 최신 경제지표와 기대보다 약하고 더딘 중국의 리오프닝(경제활동 재개) 효과 등을 반영해 한은은 오늘 금통위 회의 직후 올해 성장률 전망치를 기존 1.6%에서 1.5% 안팎으로 하향 조정할 것으로 보입니다.전문가들은 한은이 경기에 대한 눈높이를 낮추면서 동시에 금리는 올려 경기에 부담을 주는 '모순적' 판단을 내리기 쉽지 않을 것으로 보고 있습니다.한은 입장에서는 실리콘밸리은행(SVB) 사태 이후 글로벌 금융 불안이 이어지는 가운데 계속 금리 인상으로 압박하면, 취약한 저축은행이나 카드사 등에서부터 부실 문제가 터질 수 있기 때문에 금융 안정에도 신경을 쓸 수밖에 없습니다.역대 최대 폭인 1.75%p까지 벌어진 미국(5.00∼5.25%)과의 기준금리 격차가 부담스럽지만, 아직 원화 약세와 외국인 자금 유출이 본격적으로 나타나지 않았기 때문에 한은으로서는 '추가 인상 없이 버틸 수 있다'고 생각할 가능성이 큽니다.또 연방준비제도(Fed·연준)의 6월 기준금리(정책금리) 동결설에 힘이 실리면서, 한

In [37]:
# 결과
print("#TEST 특성 행렬의 크기:", source_cv.shape)
print("#BOW  행렬의 크기:", df_DTM.shape)
print("-"*80)
print("#Top 20 유사도(BOW):\n", sorted(sim_result_cv[0].round(2), reverse=True)[:20])
sim_index= (-sim_result_lsa[0]).argsort()[:20]
print('#Top 20 유사 뉴스의 인덱스(BOW):\n', sim_index)
sim_labels = [df.iloc[i].section for i in sim_index]
print('#Top 20 유사 뉴스의 카테고리(BOW):\n', sim_labels)

#TEST 특성 행렬의 크기: (1, 13452)
#BOW  행렬의 크기: (1500, 13452)
--------------------------------------------------------------------------------
#Top 20 유사도(BOW):
 [1.0, 0.75, 0.68, 0.63, 0.62, 0.58, 0.57, 0.57, 0.56, 0.56, 0.55, 0.52, 0.52, 0.5, 0.49, 0.47, 0.47, 0.47, 0.47, 0.45]
#Top 20 유사 뉴스의 인덱스(BOW):
 [260 489 302 454 425 354 322 339 448 362 290 343 273 285 397 402 255 366
 282 257]
#Top 20 유사 뉴스의 카테고리(BOW):
 [101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101]


In [32]:
#TFIDF
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=800, random_state=7) 
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(tokenizer=my_tokenizer)
print('#첫 문서의 내용:', df.main[260])
print('#첫 문서의 카테고리:', df.section[260])
#변환된 count vector와 기존 값들과의 similarity 계산
X_train_tfidf = tfidf.fit_transform(df.main) 
#X_train_lsa = svd.fit_transform(X_train_tfidf)


source= df.main[260]
source_tfidf = tfidf.transform([source])
#X_test_lsa = svd.transform(source_tfidf)
sim_result_tfidf= cosine_similarity(source_tfidf, X_train_tfidf) 

#첫 문서의 내용: ▲ 이창용 총재, 금융통화위원회 주재 사진 : 연합뉴스?한국은행 금융통화위원회(금통위)가 오늘(25일) 오전 9시 통화정책방향 회의를 열고 현재 3.50%인 기준금리의 조정 여부를 결정할 예정입니다. 경제·금융 전문가들 사이에서는 금통위가 기준금리를 2월과 4월에 이어 다시 동결할 것이라는 전망이 압도적으로 우세합니다. 지난달 소비자물가 상승률(3.7%)이 14개월 만에 3%대로 떨어져 인플레이션 압력이 다소 줄어든 상태에서, 굳이 한은이 무리한 금리 인상으로 가뜩이나 위축된 경기에 찬물을 끼얹을 이유가 없다는 분석입니다.수출 부진이 이어지는 가운데 지난 1분기 우리나라 실질 국내총생산(GDP) 성장률은 민간소비 덕에 겨우 두 분기 연속 역성장을 피했고, 3월 경상수지도 국내기업 해외 현지법인의 배당에 기대 겨우 석 달 연속 적자를 모면했습니다.하지만 통관기준 무역수지는 4월(-26억 2천만 달러)까지 14개월째 적자에서 벗어나지 못하고 있습니다.이런 최신 경제지표와 기대보다 약하고 더딘 중국의 리오프닝(경제활동 재개) 효과 등을 반영해 한은은 오늘 금통위 회의 직후 올해 성장률 전망치를 기존 1.6%에서 1.5% 안팎으로 하향 조정할 것으로 보입니다.전문가들은 한은이 경기에 대한 눈높이를 낮추면서 동시에 금리는 올려 경기에 부담을 주는 '모순적' 판단을 내리기 쉽지 않을 것으로 보고 있습니다.한은 입장에서는 실리콘밸리은행(SVB) 사태 이후 글로벌 금융 불안이 이어지는 가운데 계속 금리 인상으로 압박하면, 취약한 저축은행이나 카드사 등에서부터 부실 문제가 터질 수 있기 때문에 금융 안정에도 신경을 쓸 수밖에 없습니다.역대 최대 폭인 1.75%p까지 벌어진 미국(5.00∼5.25%)과의 기준금리 격차가 부담스럽지만, 아직 원화 약세와 외국인 자금 유출이 본격적으로 나타나지 않았기 때문에 한은으로서는 '추가 인상 없이 버틸 수 있다'고 생각할 가능성이 큽니다.또 연방준비제도(Fed·연준)의 6월 기준금리(정책금리) 동결설에 힘이 실리면서, 한

In [38]:
#결과
# Example Usage
print('#shape of tfidf matrix:', X_train_tfidf.shape)
print("-"*80)
print('#20 count score of the first review:', df_DTM[0].toarray()[0][:20]) 
#첫 리뷰의 TF-IDF 벡터 중 앞 20개 값 출력
print('#20 tfidf score of the first review:', X_train_tfidf[0].toarray()[0][:20])
print("-"*80)
print("#Top 20 유사도(TFIDF):\n", sorted(sim_result_tfidf[0].round(2), reverse=True)[:20])
sim_index= (-sim_result_lsa[0]).argsort()[:20]
print('#Top 20 유사 뉴스의 인덱스(TFIDF):\n', sim_index)
sim_labels = [df.iloc[i].section for i in sim_index]
print('#Top 20 유사 뉴스의 카테고리(TFIDF):\n', sim_labels)

#shape of tfidf matrix: (1500, 13452)
--------------------------------------------------------------------------------
#20 count score of the first review: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
#20 tfidf score of the first review: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
--------------------------------------------------------------------------------
#Top 20 유사도(TFIDF):
 [1.0, 0.76, 0.69, 0.6, 0.6, 0.54, 0.54, 0.53, 0.52, 0.51, 0.51, 0.5, 0.49, 0.46, 0.45, 0.44, 0.41, 0.41, 0.41, 0.4]
#Top 20 유사 뉴스의 인덱스(TFIDF):
 [260 489 302 454 425 354 322 339 448 362 290 343 273 285 397 402 255 366
 282 257]
#Top 20 유사 뉴스의 카테고리(TFIDF):
 [101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101]


In [34]:
##결과 추출 

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=800, random_state=7) 
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(tokenizer=my_tokenizer)
print('#첫 문서의 내용:', df.main[260])
print('#첫 문서의 카테고리:', df.section[260])
#변환된 count vector와 기존 값들과의 similarity 계산
X_train_tfidf = tfidf.fit_transform(df.main) 
X_train_lsa = svd.fit_transform(X_train_tfidf)


source= df.main[260]
source_tfidf = tfidf.transform([source])
X_test_lsa = svd.transform(source_tfidf)
sim_result_lsa= cosine_similarity(X_test_lsa, X_train_lsa) 

# Example Usage
print('LSA Converted X shape:', X_train_lsa.shape)
print('Sum of explained variance ratio: {:.3f}'.format(svd.explained_variance_ratio_.sum()))
print("-"*80)
print("#Top 20 유사도(lsa):\n", sorted(sim_result_lsa[0].round(2), reverse=True)[:20])
sim_index= (-sim_result_lsa[0]).argsort()[:20]
print('#Top 20 유사 뉴스의 인덱스(lsa):\n', sim_index)
sim_labels = [df.iloc[i].section for i in sim_index]
print('#Top 20 유사 뉴스의 카테고리(lsa):\n', sim_labels)


#첫 문서의 내용: ▲ 이창용 총재, 금융통화위원회 주재 사진 : 연합뉴스?한국은행 금융통화위원회(금통위)가 오늘(25일) 오전 9시 통화정책방향 회의를 열고 현재 3.50%인 기준금리의 조정 여부를 결정할 예정입니다. 경제·금융 전문가들 사이에서는 금통위가 기준금리를 2월과 4월에 이어 다시 동결할 것이라는 전망이 압도적으로 우세합니다. 지난달 소비자물가 상승률(3.7%)이 14개월 만에 3%대로 떨어져 인플레이션 압력이 다소 줄어든 상태에서, 굳이 한은이 무리한 금리 인상으로 가뜩이나 위축된 경기에 찬물을 끼얹을 이유가 없다는 분석입니다.수출 부진이 이어지는 가운데 지난 1분기 우리나라 실질 국내총생산(GDP) 성장률은 민간소비 덕에 겨우 두 분기 연속 역성장을 피했고, 3월 경상수지도 국내기업 해외 현지법인의 배당에 기대 겨우 석 달 연속 적자를 모면했습니다.하지만 통관기준 무역수지는 4월(-26억 2천만 달러)까지 14개월째 적자에서 벗어나지 못하고 있습니다.이런 최신 경제지표와 기대보다 약하고 더딘 중국의 리오프닝(경제활동 재개) 효과 등을 반영해 한은은 오늘 금통위 회의 직후 올해 성장률 전망치를 기존 1.6%에서 1.5% 안팎으로 하향 조정할 것으로 보입니다.전문가들은 한은이 경기에 대한 눈높이를 낮추면서 동시에 금리는 올려 경기에 부담을 주는 '모순적' 판단을 내리기 쉽지 않을 것으로 보고 있습니다.한은 입장에서는 실리콘밸리은행(SVB) 사태 이후 글로벌 금융 불안이 이어지는 가운데 계속 금리 인상으로 압박하면, 취약한 저축은행이나 카드사 등에서부터 부실 문제가 터질 수 있기 때문에 금융 안정에도 신경을 쓸 수밖에 없습니다.역대 최대 폭인 1.75%p까지 벌어진 미국(5.00∼5.25%)과의 기준금리 격차가 부담스럽지만, 아직 원화 약세와 외국인 자금 유출이 본격적으로 나타나지 않았기 때문에 한은으로서는 '추가 인상 없이 버틸 수 있다'고 생각할 가능성이 큽니다.또 연방준비제도(Fed·연준)의 6월 기준금리(정책금리) 동결설에 힘이 실리면서, 한

In [40]:
# 결과
print('#shape of tfidf matrix:', X_train_tfidf.shape)
print('LSA Converted X shape:', X_train_lsa.shape)
print('Sum of explained variance ratio: {:.3f}'.format(svd.explained_variance_ratio_.sum()))
print("-"*80)
print("#Top 20 유사도(lsa):\n", sorted(sim_result_lsa[0].round(2), reverse=True)[:20])
sim_index= (-sim_result_lsa[0]).argsort()[:20]
print('#Top 20 유사 뉴스의 인덱스(lsa):\n', sim_index)
sim_labels = [df.iloc[i].section for i in sim_index]
print('#Top 20 유사 뉴스의 카테고리(lsa):\n', sim_labels)

#shape of tfidf matrix: (1500, 13452)
LSA Converted X shape: (1500, 800)
Sum of explained variance ratio: 0.952
--------------------------------------------------------------------------------
#Top 20 유사도(lsa):
 [1.0, 0.84, 0.75, 0.65, 0.65, 0.58, 0.57, 0.57, 0.55, 0.55, 0.55, 0.53, 0.52, 0.5, 0.46, 0.45, 0.44, 0.43, 0.43, 0.42]
#Top 20 유사 뉴스의 인덱스(lsa):
 [260 489 302 454 425 354 322 339 448 362 290 343 273 285 397 402 255 366
 282 257]
#Top 20 유사 뉴스의 카테고리(lsa):
 [101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101, 101]


##### 다른 형태 결과물

In [43]:
#결과추출 다른 형태
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

cv = CountVectorizer( tokenizer=my_tokenizer)
df_DTM = cv.fit_transform(df.main) #review를 이용하여 count vector를 학습하고, 변환


# Cosine Similarity
cosine_sim = cosine_similarity(df_DTM)

# Get Recommendations
def get_recommendations(idx, cosine_sim=cosine_sim):
    # Similarity Scores
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:20]

    # Movie Indices
    movie_indices = [idx[0] for idx in sim_scores]

    # Similar Articles
    similar_articles = df[['main', 'section']].iloc[movie_indices]
    similar_articles['similarity'] = [idx[1] for idx in sim_scores]

    return similar_articles

C:\ProgramData\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [44]:
# Example Usage
goal = 260  # Index of the article you want recommendations for
print("Original Article:")
print(df.loc[goal])
print("\nRecommended Articles:")
ans = pd.DataFrame(get_recommendations(goal))
ans

Original Article:
title                        오늘(25일) 한국은행 금통위..기준금리 3연속 동결하나
date                                     2023.05.25. 오전 8:24
main       ▲ 이창용 총재, 금융통화위원회 주재 사진 : 연합뉴스?한국은행 금융통화위원회(금통...
section                                                  101
Name: 260, dtype: object

Recommended Articles:


,main,section,similarity
489,한국은행 금융통화위원회(이하 금통위)가 지난 2월과 4월에 이어 오늘(25일) 기준...,101,0.751891
302,한국은행 금융통화위원회가 지난 2월과 4월에 이어 오늘 기준금리를 다시 3.5%로 ...,101,0.677447
454,[앵커]이창용 한은 총재의 기자간담회와 질의응답은 여기까지 보겠습니다.오늘(25일)...,101,0.626653
425,한국은행이 기준금리를 3연속 동결했다. 경제성장률은 1.4%로 하향했다. ? 한...,101,0.622411
354,이창용 한국은행 총재가 25일 서울 중구 한국은행에서 열린 금융통화위원회에서 의사봉...,101,0.576824
322,"이창용 총재, 연내 인하 가능성 언급 주목성장률 전망치 1.6%→1.5% 안팎 하향...",101,0.570181
290,(상보)\n\n\n\n이창용 한국은행 총재(사진 가운데)가 25일 오전 서울 중구 ...,101,0.567113
362,■ 경제현장 오늘 '이슈체크' - 유혜미 한양대 금융경제학부 교수한국은행 금융통화위...,101,0.563269
339,이창용 한국은행 총재가 25일 서울 중구 한국은행에서 열린 금융통화위원회에서 의사봉...,101,0.559118
448,"올 성장률 1.6→1.4%로 낮춰…커지는 경기둔화 압력 고려""추가 인상은 물가 둔화...",101,0.554692


In [41]:
#결과추출 다른 형태
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
# TF-IDF
tfidf = TfidfVectorizer(tokenizer=my_tokenizer)

# SVD
X_train_tfidf = tfidf.fit_transform(df.main) 

# Cosine Similarity
cosine_sim = cosine_similarity(X_train_tfidf)

# Get Recommendations
def get_recommendations(idx, cosine_sim=cosine_sim):
    # Similarity Scores
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:20]

    # Movie Indices
    movie_indices = [idx[0] for idx in sim_scores]

    # Similar Articles
    similar_articles = df[['main', 'section']].iloc[movie_indices]
    similar_articles['similarity'] = [idx[1] for idx in sim_scores]

    return similar_articles


C:\ProgramData\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [42]:
# Example Usage
goal = 260  # Index of the article you want recommendations for
print("Original Article:")
print(df.loc[goal])
print("\nRecommended Articles:")
ans = pd.DataFrame(get_recommendations(goal))
ans

Original Article:
title                        오늘(25일) 한국은행 금통위..기준금리 3연속 동결하나
date                                     2023.05.25. 오전 8:24
main       ▲ 이창용 총재, 금융통화위원회 주재 사진 : 연합뉴스?한국은행 금융통화위원회(금통...
section                                                  101
Name: 260, dtype: object

Recommended Articles:


,main,section,similarity
489,한국은행 금융통화위원회(이하 금통위)가 지난 2월과 4월에 이어 오늘(25일) 기준...,101,0.759422
302,한국은행 금융통화위원회가 지난 2월과 4월에 이어 오늘 기준금리를 다시 3.5%로 ...,101,0.691074
454,[앵커]이창용 한은 총재의 기자간담회와 질의응답은 여기까지 보겠습니다.오늘(25일)...,101,0.604185
425,한국은행이 기준금리를 3연속 동결했다. 경제성장률은 1.4%로 하향했다. ? 한...,101,0.600766
339,이창용 한국은행 총재가 25일 서울 중구 한국은행에서 열린 금융통화위원회에서 의사봉...,101,0.543548
354,이창용 한국은행 총재가 25일 서울 중구 한국은행에서 열린 금융통화위원회에서 의사봉...,101,0.540212
322,"이창용 총재, 연내 인하 가능성 언급 주목성장률 전망치 1.6%→1.5% 안팎 하향...",101,0.533225
362,■ 경제현장 오늘 '이슈체크' - 유혜미 한양대 금융경제학부 교수한국은행 금융통화위...,101,0.523938
448,"올 성장률 1.6→1.4%로 낮춰…커지는 경기둔화 압력 고려""추가 인상은 물가 둔화...",101,0.514066
290,(상보)\n\n\n\n이창용 한국은행 총재(사진 가운데)가 25일 오전 서울 중구 ...,101,0.513108


In [35]:
#결과추출 다른 형태
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
# TF-IDF
tfidf = TfidfVectorizer(tokenizer=my_tokenizer)
#X = vectorizer.fit_transform(data['main'])

# SVD
svd = TruncatedSVD(n_components=800, random_state=7) 
X_train_tfidf = tfidf.fit_transform(df.main) 
X_train_lsa = svd.fit_transform(X_train_tfidf)

# Cosine Similarity
cosine_sim = cosine_similarity(X_train_lsa)

# Get Recommendations
def get_recommendations(idx, cosine_sim=cosine_sim):
    # Similarity Scores
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:20]

    # Movie Indices
    movie_indices = [idx[0] for idx in sim_scores]

    # Similar Articles
    similar_articles = df[['main', 'section']].iloc[movie_indices]
    similar_articles['similarity'] = [idx[1] for idx in sim_scores]

    return similar_articles


C:\ProgramData\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [46]:
# Example Usage
print('#shape of tfidf matrix:', X_train_tfidf.shape)
print('LSA Converted X shape:', X_train_lsa.shape)
print('Sum of explained variance ratio: {:.3f}'.format(svd.explained_variance_ratio_.sum()))
print("-"*80)
goal = 260  # Index of the article you want recommendations for
print("Original Article:")
print(df.loc[goal])
print("\nRecommended Articles:")
ans = pd.DataFrame(get_recommendations(goal))
ans

#shape of tfidf matrix: (1500, 13452)
LSA Converted X shape: (1500, 800)
Sum of explained variance ratio: 0.952
--------------------------------------------------------------------------------
Original Article:
title                        오늘(25일) 한국은행 금통위..기준금리 3연속 동결하나
date                                     2023.05.25. 오전 8:24
main       ▲ 이창용 총재, 금융통화위원회 주재 사진 : 연합뉴스?한국은행 금융통화위원회(금통...
section                                                  101
Name: 260, dtype: object

Recommended Articles:


,main,section,similarity
489,한국은행 금융통화위원회(이하 금통위)가 지난 2월과 4월에 이어 오늘(25일) 기준...,101,0.751891
302,한국은행 금융통화위원회가 지난 2월과 4월에 이어 오늘 기준금리를 다시 3.5%로 ...,101,0.677447
454,[앵커]이창용 한은 총재의 기자간담회와 질의응답은 여기까지 보겠습니다.오늘(25일)...,101,0.626653
425,한국은행이 기준금리를 3연속 동결했다. 경제성장률은 1.4%로 하향했다. ? 한...,101,0.622411
354,이창용 한국은행 총재가 25일 서울 중구 한국은행에서 열린 금융통화위원회에서 의사봉...,101,0.576824
322,"이창용 총재, 연내 인하 가능성 언급 주목성장률 전망치 1.6%→1.5% 안팎 하향...",101,0.570181
290,(상보)\n\n\n\n이창용 한국은행 총재(사진 가운데)가 25일 오전 서울 중구 ...,101,0.567113
362,■ 경제현장 오늘 '이슈체크' - 유혜미 한양대 금융경제학부 교수한국은행 금융통화위...,101,0.563269
339,이창용 한국은행 총재가 25일 서울 중구 한국은행에서 열린 금융통화위원회에서 의사봉...,101,0.559118
448,"올 성장률 1.6→1.4%로 낮춰…커지는 경기둔화 압력 고려""추가 인상은 물가 둔화...",101,0.554692


##### 감사합니다